In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2025
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [2]:
!pip install --upgrade cdpcli 

In [1]:
# Import inference-related libraries
import httpx
import json
import time
import os
import subprocess
from urllib.parse import urlparse, urlunparse
from typing import Optional, Dict, Any, List

In [2]:
# Used while configuring CDP credentials config
import getpass
from llmopsUtils import *

CDP config function defined!


In [5]:
### CONNECT TO REMOTE REGISTRY FOR MODEL DOWNLOAD

In [6]:
# Configure CDP control plane credentials
access_key_id = getpass.getpass("Enter your CDP access key ID: ")
private_key = getpass.getpass("Enter your CDP private key: ")
configure_cdp("cdp_access_key_id", access_key_id)
configure_cdp("cdp_private_key", private_key)

Enter your CDP access key ID:  ········
Enter your CDP private key:  ········


In [7]:
ENVIRONMENT_NAME = "nov-paul-cdp-env"

CDP_TOKEN = get_ums_jwt_token()
print("CDP TOKEN: ", CDP_TOKEN)

CDP TOKEN:  eyJraWQiOiIzYzhlNzA3OTEyZmI0NTA1ODE3NzE3YzMyOTU4MmQwMTFjYjlmNTAwIiwidHlwIjoiSldUIiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJwYXVsZGVmdXNjbyIsImF1ZCI6Imh0dHBzOi8vZGUuYTQ2NS05cTRrLmNsb3VkZXJhLnNpdGUiLCJpc3MiOiJodHRwczovL2NvbnNvbGVhdXRoLmNkcC5jbG91ZGVyYS5jb20vNTU4YmMxZDItODg2Ny00MzU3LTg1MjQtMzExZDUxMjU5MjMzIiwiZ3JvdXBzIjoiY2RwX3NhbmRib3hfd29ya2Vyc193dyBjb21wLWNsdXN0ZXJzLXRlc3RzLWF3LWNkcC1hZG1pbi1ncm91cCBzYW5kYm94LWRlZmF1bHQtc2UtYWRtaW4gY29tcC1jbHVzdGVycy10ZXN0cy1hdy1jZHAtdXNlci1ncm91cCBwYXVsLXByb2pzLW5vdi0xMS1hdy1jZHAtYWRtaW4tZ3JvdXAgcGRmLXNkYngtYXctY2RwLXVzZXItZ3JvdXAgcGRmLXNkYngtYXctY2RwLWFkbWluLWdyb3VwIHBhdWwtcHJvanMtbm92LTExLWF3LWNkcC11c2VyLWdyb3VwIF9jX2ltcGFsYV9hZG1pbnNfNmJhMTRkOGEgX2NfZGZfZGV2ZWxvcF82YmExNGQ4YSBfY19tbF91c2Vyc18zMThlMGIyYSBfY19tbF9hZG1pbnNfNWQzNTUxOGEgX2NfbWxfYWRtaW5zXzNhMjgyZjZkIF9jX21sX2FkbWluc19mNTU1YWE2IF9jX3Jhbmdlcl9hZG1pbnNfNmJhMTRkOGEgX2NfZGZfcHVibGlzaF81MDI2NjY3ZSBfY19kZV91c2Vyc182YmExNGQ4YSBfY19lbnZfcHJpdmlsZWdlZF91c2Vyc182YmExNGQ4YSBfY19kZl9hZG1pbmlzdGVyX

In [8]:
REGISTRY_ENDPOINT = get_registry_endpoint(ENVIRONMENT_NAME)
print("REGISTRY ENDPOINT: ", REGISTRY_ENDPOINT)

REGISTRY ENDPOINT:  https://modelregistry.ml-a31f9f64-7a4.nov-paul.a465-9q4k.cloudera.site


In [9]:
REGISTERED_MODEL_NAME = "onnx_classifier_notebook"

model_details = get_model_details(REGISTRY_ENDPOINT, REGISTERED_MODEL_NAME, CDP_TOKEN)
print(model_details)

{'created_at': '2025-11-25T18:13:53.946Z', 'creator': {'user_name': 'pauldefusco'}, 'id': 'lbzj-8siy-gl2y-4b88', 'name': 'onnx_classifier_notebook', 'tags': None, 'updated_at': '2025-11-25T18:13:53.946Z', 'visibility': 'private'}


In [10]:
MODEL_VERSION = get_most_recent_model_version(REGISTRY_ENDPOINT, model_details['id'], CDP_TOKEN)

In [11]:
MODEL_ID = model_details['id']
print(MODEL_ID)
print(MODEL_VERSION)

lbzj-8siy-gl2y-4b88
7


In [12]:
MODEL_INFO = list_model_info(REGISTRY_ENDPOINT, MODEL_ID, CDP_TOKEN)
print(MODEL_INFO)

{'created_at': '2025-11-25T18:13:53.946Z', 'creator': {'user_name': 'pauldefusco'}, 'id': 'lbzj-8siy-gl2y-4b88', 'model_versions': [{'artifact_uri': 's3a://nov-paul-buk-5d0c06df/data/modelregistry/lbzj-8siy-gl2y-4b88/fg2a-99e7-fy80-90vn/model.tar.gz', 'created_at': '2025-11-25T18:13:53.948Z', 'metadata': {'mlflowMetadata': {'experiment_id': '48vs-yru7-jdpt-iidm', 'metrics': [], 'params': [], 'run_id': '73pi-zsdd-r1jb-1zst', 'tags': [{'key': 'engineID', 'value': 't06g8w4jovm71tp7'}, {'key': 'mlflow.log-model.history', 'value': '[{"run_id": "73pi-zsdd-r1jb-1zst", "artifact_path": "classifier_pytorch", "utc_time_created": "2025-11-25 18:13:32.982999", "flavors": {"pytorch": {"model_data": "data", "pytorch_version": "2.9.1+cu128", "code": null}, "python_function": {"pickle_module_name": "mlflow.pytorch.pickle_module", "loader_module": "mlflow.pytorch", "python_version": "3.10.17", "data": "data", "env": {"conda": "conda.yaml", "virtualenv": "python_env.yaml"}, "config": {"device": null}}},

In [13]:
latestVersionURI = MODEL_INFO['model_versions'][-1]['artifact_uri']

In [14]:
latestVersionURI

's3a://nov-paul-buk-5d0c06df/data/modelregistry/lbzj-8siy-gl2y-4b88/tmh8-upks-jqeg-e51j/model.tar.gz'

In [20]:
### DOWNLOAD MODEL TO LOCAL

In [3]:
download_model('s3a://nov-paul-buk-5d0c06df/data/modelregistry/lbzj-8siy-gl2y-4b88/tmh8-upks-jqeg-e51j/model.tar.gz')

Bucket: nov-paul-buk-5d0c06df
Key: data/modelregistry/lbzj-8siy-gl2y-4b88/tmh8-upks-jqeg-e51j/model.tar.gz
Download successful!
Extraction complete!


In [21]:
### REGISTER THE MODEL LOCALLY

In [1]:
import mlflow
import torch
import numpy as np
import onnx
import onnxruntime as ort
import onnxmltools
from onnxmltools.convert.common.data_types import FloatTensorType

In [2]:
# Prepare input examples for model logging
# Use a small sample of test data as input example
input_example = np.array([
    [-0.66, -1.77],
    [-0.74, -1.42],
    [-1.50, -0.92]
], dtype=np.float32)

# Convert to torch.float32 tensor
input_example_tensor = torch.from_numpy(input_example).float()

In [4]:
REGISTERED_MODEL_NAME_ONNX = "onnx_classifier_notebook"
onnx_path = "/home/cdsw/onnx_model" + "/model.onnx"
with mlflow.start_run() as run:    
    # Log the ONNX model to MLflow with input example and signature  
    print("Logging ONNX model to MLflow...")
    onnx_model = onnx.load(onnx_path)
    mlflow.onnx.log_model(
        onnx_model,
        "classifier_onnx",
        registered_model_name=f"{REGISTERED_MODEL_NAME_ONNX}",
        input_example=input_example
    )
    
    print("Models logged successfully to MLflow!")
    print(f"MLflow Run ID: {run.info.run_id}")
    print("✓ Models logged with input examples and signatures!")

No experiment set using default experiment.Please set experiment using mlflow.set_experiment('<your experiment name>') to avoid using default experiment.


Logging ONNX model to MLflow...


Successfully registered model 'onnx_classifier_notebook'.
2025/11/26 18:23:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: onnx_classifier_notebook, version 1


Models logged successfully to MLflow!
MLflow Run ID: fyaq-k08m-9phj-4m5g
✓ Models logged with input examples and signatures!


Created version '1' of model 'onnx_classifier_notebook'.


In [36]:
### DEPLOY MODEL IN LOCAL INFERENCE SERVICE

##### Get CDP Token and input in CDP_TOKEN variable below.
```
cdp --endpoint-url https://your.endpoint.url.com/ \
  --no-verify-tls \
  --form-factor private iam generate-workload-auth-token \
  --workload-name DE
```

In [18]:
import os
import httpx
import ssl

CAII_DOMAIN = "ml-f884fd9f-fe8.apps.cdppvc.ares.olympus.cloudera.com" #os.envion["INFERENCE_SERVICE_DOMAIN"] # Obtain from the Inference Service UI.
ENDPOINT_NAME = "my-pytorch-clf" # Arbitrarily assigned by user.
MODEL_ID = "jjie-jyra-xivj-baet" # Obtain from the AI Registry UI.
MODEL_VERSION = 1 # Obtain from the AI Registry UI. Default to 1 if you've only registered the model once.
CDP_TOKEN = ".VMUtY8XMfOuk94Zemh5H9cseMps9JiJ2kf1TOKfbfm-" #os.environ["CDP_TOKEN"]

In [21]:
def deploy_model_to_caii(caii_domain, cdp_token, model_id, model_version, endpoint_name):
    # Method to deploy an AI Registry model to AI Inference Service

    deploy_url = f"https://{caii_domain}/api/v1alpha1/deployEndpoint"
    
    REQUESTS_CA_BUNDLE="/etc/ssl/certs/ca-certificates.crt"    
    headers = {'Authorization': 'Bearer ' + cdp_token,
           'Content-Type': 'application/json'}

    ctx = ssl.create_default_context(cafile=os.environ.get("REQUESTS_CA_BUNDLE"))
    client = httpx.Client(headers=headers, verify=ctx)
    
    r = client.get(os.environ["MODEL_REGISTRY_URL"]+'/api/v2/models')
    
    # Deploy the model endpoint. Note that "serving-default" is the only valid
    # namespace. Adjust resources and autoscaling parameters as you need. Also note
    # that we're not requesting a GPU for the model deployment. If your model requires GPUs,
    # you can add it to the "resources" section, e.g.
    # "resources": {
    #     "num_gpus": "2",
    #     "req_cpu": "4",
    #     "req_memory": "8Gi"
    #  }
    #
    deploy_payload = {
        "namespace": "serving-default",
        "name": f"{endpoint_name}",
        "source": {
            "registry_source": {
                "model_id": f"{model_id}",
                "version": f"{model_version}"
            }
        },
        "resources": {
            "req_cpu": "2",
            "req_memory": "4Gi",
            "num_gpus": "0"
        },
        "autoscaling": {
            "min_replicas": "1",
            "max_replicas": "2"
        }
    }
    try:
        response = client.post(deploy_url, json=deploy_payload)
        response.raise_for_status()
        print(f"Deployed {endpoint_name} successfully!")
    except httpx.HTTPStatusError as e:
        print(f"HTTP {e.response.status_code}: {e.response.text}")
    except httpx.RequestError as e:
        print(f"Error deploying {endpoint_name}: {e}")

In [22]:
deploy_model_to_caii(CAII_DOMAIN, CDP_TOKEN, MODEL_ID, MODEL_VERSION, ENDPOINT_NAME)

Deployed my-pytorch-clf successfully!
